### 🔍 Objective:
This project will **introduce you to CNNs**.

### 📌 What You’ll Do:
1. Define suitable transforms/augmentations for your `train` and `test` images.
2. Pass these images into PyTorch `DataLoaders` for batch processing.
3. Implement `CNN` class architecture for pneumonia image classification.
4. Train and validate your model.

💡 **PLEASE PLEASE PLEASE look things up!!! This is YOUR learning experience.**

---

In [3]:
import torch
import kagglehub
import torch.nn as nn

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

train_path = f'{path}/chest_xray/train/'
test_path = f'{path}/chest_xray/test/'

c:\Users\Kadin Wolfe\Downloads\project-2-cnns-w-lfe\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.29G/2.29G [00:45<00:00, 54.4MB/s]

Extracting files...


#### 📌 ***TASK 1 - DATA PREPROCESSING***

Define image augmentations in the cell below using two variables:  

- **`transform_train`**: Stores transforms for training images. You can include any augmentations you prefer.  
- **`transform_test`**: Stores transforms for your test images. As a best practice, limit these transformations to only the essential ones from `transform_train`.

Lastly, be sure to convert all images to [tensors](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) via the `transforms.ToTensor()` transform. Don't know transforms? [Click here](https://pytorch.org/vision/stable/transforms.html).

In [4]:
from torchvision import transforms

# augmentations for training
transform_train = transforms.Compose([
    # resize & crop to a fixed size (e.g. 224×224)
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    # random flips/rotations for robustness
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    # slight brightness/contrast jitter
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    # convert PIL→Tensor and normalize (here for single-channel x-rays)
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

# “minimal” transforms for validation/testing
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

Now, we'll pass our images and transforms into a `DataLoader`, which allows us to train our model in batches.
Most of the code is done for you, but [click here](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) to learn more.

In [6]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# how many images per batch
batch_size = 32  

# create datasets, pointing at your folders and applying the transforms
train_dataset = ImageFolder(root=train_path, transform=transform_train)
test_dataset  = ImageFolder(root=test_path,  transform=transform_test)

# wrap them in DataLoaders for iteration
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,        # shuffle for training
    num_workers=4,       # parallel data loading (tweak as your machine allows)
    pin_memory=True,     # often speeds up GPU training
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,       # keep order consistent for evaluation
    num_workers=4,
    pin_memory=True,
)

#### 📌 ***TASK 2 - CNN Architecture***

This is where you have all the creative freedom in the world. Here are some good questions to ask yourself:

- How many [channels](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-49AG77e4Qp2e7EkARdFsTA) should go into the input layer?
- What measures can I take to avoid [overfitting](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-YdAbhqQzRZaEq39BEQzA6w)?
- What matters to me? (Training Speed / Performance tradeoffs)
- **CONVOLUTION. ACTIVATION FUNCTION. POOLING!!!** 📢📢📢

Not comfortable with PyTorch? [Here](https://youtu.be/mozBidd58VQ?si=TE2_81TEQko1eDXT). Go and make me the best [CNN](https://www.datacamp.com/tutorial/introduction-to-convolutional-neural-networks-cnns) I've ever seen :)

In [15]:
import torch
import torch.nn as nn


image_height = 224
image_width  = 224

import torch
import torch.nn as nn

class PneumoniaCNN(nn.Module):
    def __init__(self, num_classes: int = 2):
        super().__init__()
        # input is now 3-channel RGB
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),              # → 16×112×112

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),              # → 32×56×56

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),              # → 64×28×28
        )

        # 64 channels, each 28×28 after three pools
        flattened_dim = 64 * 28 * 28

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(flattened_dim, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



#### 📌 ***TASK 3 - DEFINE TRAIN FUNCTION***  

Define `process_forward_phase` and `train` to update model weights with each new [epoch/iteration](https://www.perplexity.ai/search/i-m-a-curious-naiss-mlb-studen-7SJECNYrS1iYxUR032dp7A). Here are the steps:

- **Forward pass:** Here, our batch is taken through the network to output a prediction (Normal/Pneumonia)
- **Backward pass:** The model goes "What's our loss? Hmmm... Not quite what I want. This means my `weights` aren't adjusted properly. Let me propagate my `loss` backward in hopes of correcting my weights."

We use **`f1_score`** as the primary metric and also display **`accuracy`** for comparison. Most steps are outlined for you—just follow the structure provided!

In [20]:
import torch
from tqdm import tqdm # Visualize training progress
from sklearn.metrics import f1_score, accuracy_score

def process_forward_pass(model, batch, criterion, device):
    """
    Runs one forward pass and returns:
      - loss (torch.Tensor),
      - predictions (torch.Tensor on CPU, shape [batch]),
      - labels    (torch.Tensor on CPU, shape [batch]).
    """
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    # 1) forward
    outputs = model(images)                 # shape [B, num_classes]

    # 2) compute loss
    loss = criterion(outputs, labels)       # CrossEntropyLoss expects labels as long

    # 3) get predicted class (argmax over logits)
    preds = torch.argmax(outputs, dim=1)    # shape [B]

    # bring preds & labels back to CPU so .numpy() works downstream
    return loss, preds.detach().cpu(), labels.detach().cpu()


def train(model, train_loader, criterion, optimizer, epochs, device):
    """
    Loops over `epochs` and for each batch:
      - zeros grads
      - does forward pass
      - backward + optimizer.step
      - accumulates preds & labels for metrics
    Prints loss/acc/F1 at the end of each epoch.
    """
    model.to(device)

    for epoch in range(epochs):
        model.train()
        all_preds, all_labels = [], []

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in loop:
            optimizer.zero_grad()

            # forward → loss, preds, labels
            loss, preds, labels = process_forward_pass(model, batch, criterion, device)

            # backward → update weights
            loss.backward()
            optimizer.step()

            # collect for metrics
            # after:
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())


            # update the tqdm bar with current loss
            loop.set_postfix(loss=loss.item())

        # compute epoch metrics
        acc = accuracy_score(all_labels, all_preds)
        f1  = f1_score(all_labels, all_preds)

        print(f"Epoch {epoch+1}/{epochs} →  Acc: {acc*100:.2f}%,  F1: {f1:.4f}")


After your model trains, you want to see how well it performs on **unseen data.** Meaning, if this were a live hospital NEEDING your predictions to classify patients with pneumonia, how well would it do?


You simply have to run this cell; all the code is implemented for you (Assuming `process_forward_phase` works fine). 😊

In [25]:
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

def test(model, test_loader, criterion, device):
    model.to(device)
    model.eval()

    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        loop = tqdm(test_loader, desc="Testing")
        for batch in loop:
            # call the forward-pass helper you actually defined
            loss, preds, labels = process_forward_pass(model, batch, criterion, device)
            total_loss += loss.item()

            # no NumPy needed
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    # aggregate metrics
    avg_loss = total_loss / len(test_loader)
    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f} | Acc: {acc*100:.2f}% | F1: {f1:.4f}")


#### 📌***TASK 4 - TRAIN MODEL***

We're close!!! We simply need to instantiate the `model`, define a suitable `criterion` (loss), and use an `optimizer` (thing to speed up backpropagation).

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1) Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) Instantiate 
model = PneumoniaCNN(num_classes=2).to(device)

# 3) Define loss
criterion = nn.CrossEntropyLoss()

# 4) optimizer
optimizer = optim.Adam(
    model.parameters(),
    lr=1e-3,         # start with 0.001
    weight_decay=1e-4  # small L2 regularization to help generalization
)

# 5) Choose how many epochs you want
epochs = 10

# 6) Run the training loop
train(
    model,
    train_loader,
    criterion,
    optimizer,
    epochs=epochs,
    device=device
)


Epoch 1/10: 100%|██████████| 163/163 [01:23<00:00,  1.94it/s, loss=0.237]


Epoch 1/10 →  Acc: 82.25%,  F1: 0.8888


Epoch 2/10:   0%|          | 0/163 [00:00<?, ?it/s]c:\Users\Kadin Wolfe\Downloads\project-2-cnns-w-lfe\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 2/10: 100%|██████████| 163/163 [01:24<00:00,  1.92it/s, loss=0.188] 


Epoch 2/10 →  Acc: 88.78%,  F1: 0.9261


Epoch 3/10:   0%|          | 0/163 [00:00<?, ?it/s]c:\Users\Kadin Wolfe\Downloads\project-2-cnns-w-lfe\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 3/10: 100%|██████████| 163/163 [01:24<00:00,  1.92it/s, loss=0.19]  


...

Epoch 8/10 →  Acc: 93.35%,  F1: 0.9553


Epoch 9/10:   0%|          | 0/163 [00:00<?, ?it/s]c:\Users\Kadin Wolfe\Downloads\project-2-cnns-w-lfe\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 9/10: 100%|██████████| 163/163 [01:26<00:00,  1.89it/s, loss=0.359] 


Epoch 9/10 →  Acc: 93.67%,  F1: 0.9574


Epoch 10/10:   0%|          | 0/163 [00:00<?, ?it/s]c:\Users\Kadin Wolfe\Downloads\project-2-cnns-w-lfe\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 10/10: 100%|██████████| 163/163 [01:27<00:00,  1.86it/s, loss=0.125] 

Epoch 10/10 →  Acc: 93.87%,  F1: 0.9588


Last step: evaluate your model's performance. Remember, you get **1,000,000** brownie points 🍫 if you beat Adam's **`f1_score:`0.8549**.

In [27]:
test(model, test_loader, criterion, device)


Testing:   0%|          | 0/20 [00:00<?, ?it/s]c:\Users\Kadin Wolfe\Downloads\project-2-cnns-w-lfe\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Testing: 100%|██████████| 20/20 [00:25<00:00,  1.25s/it]

Test Loss: 2.6899 | Acc: 66.19% | F1: 0.7867
